In [ ]:
#default imports 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import os
#settings 
pd.set_option("display.max_columns", None)

In [ ]:
#get the data 
df1 = pd.read_csv("../../data/raw/ks-projects-201801.csv")
df2 = pd.read_csv("../../data/raw/ks-projects-201612.csv", encoding='latin-1')

In [ ]:
print(df1.shape)
df1.head()

So we have... "to ignore":
* ID (for now)
* name (is this relevant?)

we have as categorical: 
* category (159)...
* main category (15) --- typo in "journalism", correct
* country (23) --- typo for "N,0"
* currency (14) - looks fine --- does it matter? all numericals are in USD - ah, the goal seems to be in currency. hence the double column. good! 

In [ ]:
print(df1['category'].nunique())
display(df1['category'].unique())
print(df1['main_category'].nunique())
display(df1['main_category'].unique())
#go for the country
print(df1['country'].nunique())
display(df1['country'].unique())
#and the currency 
print(df1['currency'].nunique())
display(df1['currency'].unique())
#and finally our target - the state 
print(df1['state'].nunique())
print(df1['state'].unique())

### first view "target"
it might make sense to predict individual outcomes in different ways. 

Like - having the target be "successful" and basically everything else COULD work 

however, looking into it a bit deeper would also be helpful. What does "live" mean? 

We have as duration
* launched (str) --- convert to datetime
* deadline (str) --- convert to datetime

As true numericals: 
* goal (BUT as it's in CURRENCY maybe drop that)
* pledged (same - probably in currency?)
* usd pledged 
* usd_pledged_real --- how are these two not the same? 
* usd_goal_real - what was wanted 
and finally 
* backers (number )

In [ ]:
df1.describe().T

### Check for Null values

In [ ]:
df1.duplicated().value_counts

In [ ]:
df1.isna().sum()
#ok so some have no name - doesn't matter 
#and only usd_pledged has null, but not "pledged_real" so maybe just ignore the pledged one... 

### preliminary cleanup

In [ ]:
df1['deadline'] = pd.to_datetime(df1.deadline, format='%Y-%m-%d')
df1['launched'] = pd.to_datetime(df1.launched, format='%Y-%m-%d %H:%M:%S')
#df1.info()
df1.head(20)

Some ideas for new features: 
* duration (if between launched and deadline) - vs. "still running...?"
* consolidate "state" into three - successful, unsuccessful, undefined 
* --- then we could work with two and drop "undefined" for now 
then ... 
* (DONE) Outliers cleaning for backers at 100.000 (cutoff) - there's not that many - ok let's take 80.000
* 

### small checkout

In [ ]:
#sns.catplot(data=df1.sort_values("state"), x='state', y=)
sns.histplot(data=df1['state'], bins = 200)

In [ ]:
sns.jointplot(data=df1, x='backers', y='usd_pledged_real')
#so far so obvious

In [ ]:
#sns.boxplot(data=df1['backers'])
#ah, so that's up 
#sns.catplot(data=df1.query('backers > 0'), x='backers', y='usd_pledged_real')
#what's with this column..? 
df1['backers'].describe()
#I'd cut at 100.000 or else ... 

In [ ]:
# dftry = df1.query('backers > 100')
# display(df1.shape)
# dftry.shape
df1_test = df1.query("backers < 80.000")
df1_test['backers'].plot(kind='hist', bins=100)
#there you go - better ... clearest power law ever ... 

In [ ]:
#df1_test['usd_goal_real'].plot(kind='box')
#ya think... 
sns.catplot(data=df1_test, x="state", y="backers", kind="boxen")
#ah! now we're talking!

In [ ]:
df1_test['usd_pledged_real'].sort_values().tail(30)
df1_test2 = df1_test.query('usd_pledged_real < 100000 & usd_pledged_real > 2000')
sns.catplot(data=df1_test2, x="state", y="usd_pledged_real", kind="boxen")
#ok another one of these pure power laws... 
# we reach a "normal" view around 10.000 - so I'd go for bins 
#ah but do you see the suspended...?

In [ ]:
df1_test['usd_goal_real'].max() #wait, that can't be... 166361390.71 ??? thats hundreds of millions
print(df1_test['usd_goal_real'].min())
#let's do this differently 
df1_testsuccess = df1.query('state == "successful"')
df1_testsuccess['usd_goal_real'].sort_values().tail(30) #so much better - there's the line at 1.000.000 though - and it goes down rapidly, 775000 is right behind it... 
df1_test3 = df1_testsuccess.query('usd_goal_real < 700000 & usd_goal_real > 2000')
sns.catplot(data=df1_test2, x="state", y="usd_pledged_real", kind="boxen") #oh dear... it's worse than I thought 

In [ ]:
#let's look at the main categories for a sec 
#sns.catplot(data=df1, x=df1['main_category'].value_counts(), y=df1['state'].value_counts(), kind='bar')
display(df1['main_category'].value_counts())
df1.groupby('main_category')['state'].value_counts().plot(kind='hist') #not helping yet ;) 

In [ ]:
df1['duration'] = df1['deadline'] - df1['launched']
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(8, 12))
ax.plot(df1['duration'])
ax.format_xdata = mdates.DateFormatter('%d')
plt.show()

## Final decision what to do

| Column | Cleanup operation | Comment |
|:--------:|:--------:|:--------:|
|  Row 1   |  Row 1   |  Row 1   |
|  Row 2   |  Row 2   |  Row 2   |
|  Row 3   |  Row 3   |  Row 3   |